In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sgqlc.endpoint.http import HTTPEndpoint
from time import sleep
import requests
import json


""" config.py """
API = 'https://etherlink-bridge-ghostnet.dipdup.net/v1/graphql'

In [2]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 200)
sns.set_style('whitegrid')
colors = sns.color_palette()

In [3]:
endpoint = HTTPEndpoint(API)

### Deposits:

### Withdrawals:

In [4]:
from string import Template
from typing import Iterator, List, Optional
import time


def iter_pages_fetch(
    endpoint: HTTPEndpoint,
    table_name: str,
    query_content: str,
    limit = 1000,
    max_queries: Optional[int] = None,
    delay = 1,
) -> Iterator[List]:

    query_template = Template("""
        query Query {
            $table_name(limit: $limit, offset: $offset) {
                $query_content
            }
        }
    """)
    offset = 0
    queries_count = 0

    print(f'Requesting `{table_name}`: ', end='')
    while True:
        queries_count += 1
        if queries_count > max_queries:
            break

        variables = dict(limit=limit, offset=offset)
        query = query_template.substitute(
            table_name=table_name,
            query_content=query_content,
            limit=limit,
            offset=offset
        )
        response = endpoint(query)
        if 'errors' in response:
            print('Catched errors while loading data:')
            import pdb; pdb.set_trace()
        print('x', end='')
        data = response['data'][table_name]
        if len(data):
            yield data
            offset += limit
            time.sleep(delay)
        else:
            break
    print(', done')
    return


def fetch_all(
    endpoint: HTTPEndpoint,
    table_name: str,
    query_content: str,
    max_queries: Optional[int]=None,
) -> pd.DataFrame:
    return pd.concat(
        pd.json_normalize(data)
        for data in iter_pages_fetch(
            endpoint=endpoint,
            table_name=table_name,
            query_content=query_content,
            max_queries=max_queries,
        )
        if len(data)
    )


def fetch_withdrawals(endpoint: HTTPEndpoint):
    return fetch_all(
        endpoint=endpoint,
        table_name='bridge_withdrawal',
        query_content='''
            created_at
            id
            l1_transaction_id
            l2_transaction_id
            outbox_message_id
            updated_at
        ''',
    )

In [5]:
def fetch_bridge_operations(endpoint: HTTPEndpoint, max_queries: Optional[int]=None):
    return fetch_all(
        endpoint=endpoint,
        table_name='bridge_operation',
        query_content='''
            created_at
            id
            is_completed
            is_successful
            l1_account
            l2_account
            status
            type
            updated_at
            withdrawal {
              created_at
              id
              l1_transaction_id
              l1_transaction {
                counter
                id
                initiator
                level
                nonce
                operation_hash
                outbox_message_id
                sender
                target
                timestamp
              }
              outbox_message_id
              updated_at
              l2_transaction_id
              outbox_message {
                cemented_at
                cemented_level
                commitment_id
                created_at
                id
                index
                level
                message
                parameters_hash
                proof
                updated_at
              }
              l2_transaction {
                address
                amount
                id
                kernel_withdrawal_id
                l1_account
                l1_ticket_owner
                l2_account
                l2_ticket_owner
                l2_token {
                  decimals
                  id
                  name
                  symbol
                  ticket_hash
                }
                level
                log_index
                parameters_hash
                ticket {
                  hash
                  outbox_interface
                  metadata
                  ticket_id
                  ticketer_address
                  whitelisted
                  token_id
                }
                ticket_hash
                timestamp
                token_id
                transaction_hash
                transaction_index
              }
            }
            deposit {
              created_at
              id
              inbox_message_id
              l1_transaction_id
              l2_transaction_id
              updated_at
              inbox_message {
                created_at
                level
                index
                message
                parameters_hash
                type
                updated_at
              }
              l1_transaction {
                amount
                counter
                id
                initiator
                l1_account
                l2_account
                level
                nonce
                operation_hash
                parameters_hash
                sender
                target
                ticket_hash
                timestamp
                ticket {
                  hash
                  metadata
                  ticket_id
                  ticketer_address
                  token_id
                  whitelisted
                  token {
                    contract_address
                    decimals
                    id
                    name
                    symbol
                    token_id
                    type
                  }
                }
              }
              l2_transaction {
                address
                amount
                id
                inbox_message_index
                inbox_message_level
                l2_account
                l2_token {
                  decimals
                  id
                  name
                  symbol
                  ticket_hash
                }
                level
                log_index
                ticket_hash
                ticket_owner
                timestamp
                token_id
                transaction_hash
                transaction_index
              }
            }
        ''',
        max_queries=max_queries,
    )

In [6]:
# df = fetch_withdrawals(endpoint)
df = fetch_bridge_operations(endpoint, max_queries=1000)

Requesting `bridge_operation`: xxxxxxxxxxxxxxx, done


/tmp/ipykernel_9184/723557464.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(


In [8]:
len(df)

13826

In [9]:
df_copy = df.copy()

### processing:

In [10]:
df = df_copy.copy()

datetime_fields = [
    'created_at',
    'updated_at',
    'deposit.created_at',
    'deposit.updated_at',
    'deposit.inbox_message.created_at',
    'deposit.inbox_message.updated_at',
    'deposit.l1_transaction.timestamp',
    'deposit.l2_transaction.timestamp',
    'withdrawal.created_at',
    'withdrawal.updated_at',
    'withdrawal.l2_transaction.timestamp',
    'withdrawal.outbox_message.cemented_at',
    'withdrawal.outbox_message.created_at',
    'withdrawal.outbox_message.updated_at',
    'withdrawal.l1_transaction.timestamp',
]

for field in datetime_fields:
    df[field] = pd.to_datetime(df[field])

id_to_str = lambda v: str(int(v)) if pd.notnull(v) else ''
df['deposit.inbox_message_id'] = df['deposit.inbox_message_id'].map(id_to_str)

df = df.drop(['withdrawal', 'deposit'], axis=1)

In [74]:
# some checks:
deposits = df[ df['type'] == 'deposit' ]
withdrawals = df[ df['type'] == 'withdrawal' ]
assert (deposits['deposit.l1_transaction.l1_account'] == deposits['l1_account']).all()
assert (deposits['deposit.l1_transaction.l2_account'] == deposits['l2_account']).all()
assert (withdrawals['withdrawal.l2_transaction.l1_account'] == withdrawals['l1_account']).all()
assert (withdrawals['withdrawal.l2_transaction.l2_account'] == withdrawals['l2_account']).all()

In [12]:
df['deposit.inbox_message.type'].value_counts()

deposit.inbox_message.type
transfer    12300
Name: count, dtype: int64

- [ ] `withdrawal.l1_transaction`, `withdrawal.outbox_message` is empty, why?

# Anaylyzis:

### Deposit Ticketers:

In [16]:
ticket_names = {
    # ghostnet:
    'KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m': 'xtz_native',
    'KT1S6Nf9MnafAgSUWLKcsySPNFLUxxqSkQCw': 'TEST_USDt_141',
    'KT1H7if3gSZE1pZSK48W3NzGpKmbWyBxWDHe': 'TEST_tzBTC_141',
    'KT1DjH84P98gKFptqiAevj1qoKqxrTaG1T7i': 'YOU',
    'KT1Cw8WZLp4XUPLrDWUjwwpShzCRqzRByyVh': 'TEST_SIRS_141',
    'KT1PGXCy9xkkASAXYLgp5a9szX3P4xPdHk84': 'TST12',
    'KT1CtuZZV6vnzhdn7h3t43cfGZXpcRKNvyxZ': 'TPM',
}

df['deposit.inbox_message.message.LL.ticket.address'].map(ticket_names).value_counts()

deposit.inbox_message.message.LL.ticket.address
xtz_native        12253
TEST_USDt_141        16
YOU                  12
TEST_tzBTC_141       12
TEST_SIRS_141         4
TST12                 2
TPM                   1
Name: count, dtype: int64

In [89]:
df['withdrawal.l2_transaction.l1_ticket_owner'].map(ticket_names).value_counts()

withdrawal.l2_transaction.l1_ticket_owner
xtz_native        926
TEST_USDt_141     581
YOU                 8
TEST_SIRS_141       6
TEST_tzBTC_141      3
Name: count, dtype: int64

In [17]:
df['deposit.l1_transaction.ticket.token_id'].value_counts()

deposit.l1_transaction.ticket.token_id
xtz                                       12255
KT1V2ak1MfNd3w4oyKD64ehYU7K4CrpUcDGR_0       16
KT1VyYTej9iHeAfpCKBEdpqiKaHAk4hUN7h8_0       12
KT1HmyazXfKDbo8XjwtWPXcoyHcmNPDCvZyb_0       12
KT1TCTpXXbpnWBZ8whqExokbKfUrUW3nAXDJ_0        4
KT1SekNYSaT3sWp55nhmratovWN4Mvfc6cfQ_0        2
KT1N5CWnvabCM71eBmzEhotnQX3eciLLyv8v_0        1
Name: count, dtype: int64

In [18]:
df['deposit.l1_transaction.ticket.whitelisted'].value_counts()

deposit.l1_transaction.ticket.whitelisted
True    12302
Name: count, dtype: int64

In [19]:
df['deposit.l1_transaction.ticket.token.type'].value_counts()

deposit.l1_transaction.ticket.token.type
native    12255
FA2          29
FA1.2        18
Name: count, dtype: int64

### Operations by days:

In [20]:
df.set_index('created_at').groupby(pd.Grouper(freq='D')).type.value_counts().unstack().fillna(0)

type,deposit,withdrawal
created_at,,
2024-08-27 00:00:00+00:00,5.00,1.00
2024-08-28 00:00:00+00:00,1.00,0.00
2024-08-29 00:00:00+00:00,2.00,2.00
2024-08-30 00:00:00+00:00,5.00,0.00
2024-08-31 00:00:00+00:00,2.00,0.00
2024-09-02 00:00:00+00:00,1.00,0.00
2024-09-03 00:00:00+00:00,0.00,1.00
2024-09-04 00:00:00+00:00,6.00,3.00
2024-09-05 00:00:00+00:00,"12,209.00",0.00


### Account stats:

In [23]:
df.groupby('l1_account').type.value_counts().unstack().fillna(0).sort_values('deposit', ascending=False)

type,deposit,withdrawal
l1_account,,
tz1g2RbQtxZRHw8oj4oSnhRzy7iRA2hxU4TD,"6,400.00",0.00
tz1R4EjaQqeBhBP2dztahpkzHTZ61PSssqkh,"5,801.00",0.00
tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc,49.00,605.00
tz1TZDn2ZK35UnEjyuGQRVeM2NC5tQScJLpQ,12.00,0.00
tz1KhnTgwoRRALBX6vRHRnydDGSBFsWtcJxc,5.00,5.00
tz1QCVQinE8iVj1H2fckqx6oiM85CNJSK9Sx,4.00,0.00
tz1bCRGKwwhf3DQqADACbv93sJUPVB5ChFqa,3.00,0.00
tz1fWLoHsRYx2pWx17kmuaTyuHvQbtvLj6Tw,3.00,0.00
tz1Lzmg6AD59nHMwuErfGhS3GfsPE8rcr779,2.00,1.00


In [26]:
l2_contract_names = {
    '2a0030903e5313bf145d8631e87aac5237f9ad26': 'BulkWithdrawalHelper: 450',
    'daa58536ebccb421331aae8334cd73a12a54654e': 'OutboxEnumerator'
}

df.groupby('l2_account').type.value_counts().unstack().fillna(0).sort_values('withdrawal', ascending=False)

type,deposit,withdrawal
l2_account,,
2a0030903e5313bf145d8631e87aac5237f9ad26,2.00,900.00
daa58536ebccb421331aae8334cd73a12a54654e,1.00,450.00
865e2887bafe9bdd5e68d0cc5fb9eeb9dce535fa,1.00,100.00
fa34004dda5a87a40ecef2714101472961c17560,21.00,14.00
befd2c6ffc36249ebebd21d6df6376ecf3bac448,20.00,10.00
1cf25b150d52913ad102a32c5554358f1c6f4a7e,1.00,10.00
4a338a3cb0017a3e0e9156397d98b97795032c72,3.00,9.00
0e00cfbccf7de8c0c6139fae80449bd0783599ab,2.00,5.00
7e6f6ccfe485a087f0f819eabfdbfb1a49b97677,6.00,5.00


### ERC20:
- [ ] TODO: manually check and classify all of the addresses

In [27]:
df['deposit.l2_transaction.ticket_owner'].value_counts()

deposit.l2_transaction.ticket_owner
xtz                                         5855
f68997ecc03751cb99b5b36712b213f11342452b      11
59118d19848baca5c3dd1603acfc99c4cbccc3db       7
4a338a3cb0017a3e0e9156397d98b97795032c72       3
8e73ae3cf688fbd8368c99520d26f9ef1b4d3bca       3
baa233e2f62f45e9d91dacd3d6c6a57bc2cbc575       2
8f31fa683c8dedbe61fc772b2e0a29c35069ff9d       1
Name: count, dtype: int64

In [88]:
df['withdrawal.l2_transaction.l2_token.id'].value_counts()

withdrawal.l2_transaction.l2_token.id
xtz                                         926
f68997ecc03751cb99b5b36712b213f11342452b    572
59118d19848baca5c3dd1603acfc99c4cbccc3db      8
baa233e2f62f45e9d91dacd3d6c6a57bc2cbc575      6
8e73ae3cf688fbd8368c99520d26f9ef1b4d3bca      3
Name: count, dtype: int64

### Statuses:

In [28]:
df.status.value_counts()

status
FAILED_INBOX_MATCHING_TIMEOUT             6420
FINISHED                                  5904
CREATED                                   1498
FAILED_INVALID_ROUTING_INFO_REVERTABLE       4
Name: count, dtype: int64

### Timestamps:

In [29]:
sec_in_day = 60*60*24
df['lifetime_days'] = (df['updated_at'] - df['created_at']).dt.total_seconds() / sec_in_day
df.lifetime_days.describe()

count   13,826.00
mean        16.83
std          4.49
min          0.00
25%         18.40
50%         18.40
75%         18.41
max         27.97
Name: lifetime_days, dtype: float64

### Deposit routing data:

In [30]:
df['deposit.inbox_message.message.LL.bytes'].dropna().map(len).value_counts()

deposit.inbox_message.message.LL.bytes
40    12253
80       47
Name: count, dtype: int64

### Rollup:

In [31]:
df['deposit.l1_transaction.target'].value_counts()

deposit.l1_transaction.target
sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg    12302
Name: count, dtype: int64

### Withdrawals:

In [40]:
df['withdrawal.l2_transaction.address'].value_counts()

withdrawal.l2_transaction.address
0000000000000000000000000000000000000000    1524
Name: count, dtype: int64

- [ ] TODO: analyze amounts?

In [46]:
df['withdrawal.l2_transaction.amount'].dropna().iloc[0]

'1000000000000000'

### Withdrawal ids:

In [58]:
withdrawal_ids = df['withdrawal.l2_transaction.kernel_withdrawal_id'].dropna().sort_values().astype(int)
assert not withdrawal_ids.duplicated().any()

withdrawal_ids

536       1
543       2
537       3
538       4
136       5
       ... 
423    1522
430    1523
445    1524
447    1525
448    1526
Name: withdrawal.l2_transaction.kernel_withdrawal_id, Length: 1524, dtype: int64

### Missing withdrawals:

In [69]:
# withdrawal_ids
set(range(1, withdrawal_ids.max() + 1)) - set(withdrawal_ids.to_list())

{1058, 1065}

In [95]:
df[
    df['withdrawal.l2_transaction.kernel_withdrawal_id'].isin([1057, 1059])
]['withdrawal.l2_transaction.level'].astype(int).to_list()

[6265791, 6271358]

### Levels:

In [96]:
df['withdrawal.l2_transaction.level'].value_counts()

withdrawal.l2_transaction.level
6,274,209.00    450
6,263,219.00    450
6,263,311.00    450
6,263,059.00    100
6,262,977.00     10
6,262,938.00      5
6,272,853.00      3
6,262,913.00      3
6,271,358.00      3
6,273,681.00      3
5,264,485.00      1
5,111,151.00      1
5,351,776.00      1
5,588,406.00      1
5,382,917.00      1
5,351,679.00      1
5,307,434.00      1
5,307,509.00      1
5,307,538.00      1
5,384,020.00      1
5,384,385.00      1
6,748,543.00      1
5,382,535.00      1
5,382,583.00      1
5,039,150.00      1
5,610,374.00      1
5,602,953.00      1
5,588,428.00      1
5,597,662.00      1
5,598,324.00      1
5,598,237.00      1
5,598,447.00      1
5,600,838.00      1
5,597,495.00      1
5,111,103.00      1
5,643,944.00      1
6,958,905.00      1
7,058,957.00      1
7,311,063.00      1
7,050,148.00      1
7,056,725.00      1
6,060,621.00      1
5,999,549.00      1
5,747,366.00      1
5,747,322.00      1
5,718,707.00      1
5,714,307.00      1
5,714,294.00      1
5,684,98

### What are parameters hash?

In [98]:
df['withdrawal.l2_transaction.parameters_hash'].value_counts()

withdrawal.l2_transaction.parameters_hash
16b767c43aba58568e9dc05a1e2ba47d    99
5efa431a16915f9eb900d9ec53dbde37     1
73a00a60477e59829ca035a878b48f15     1
3ea4a7662a0a56e1ae181927aeb5d6a4     1
1f155ba44ee45038907a83aa283e044a     1
90a2b27e57975864a0846c17cb3105b3     1
ea287c808271515eb6c99be4eae352b0     1
c6d79aa4d7ff5eb68df143b6c13d6134     1
Name: count, dtype: int64

### L2 tickets:

In [99]:
df['withdrawal.l2_transaction.ticket.hash'].value_counts()

withdrawal.l2_transaction.ticket.hash
71633053619282981145548614114746933288401055647068660200262395418774065381735    926
9127002238795696064282543683188674942190822398625278725945646631384514161455     581
12264034381471993363263869037032235995742822566750644554529177447255212056362      8
94307383278737158402120131047685288725437552599737436826370621448931166249308      6
59127759903831710082238055012661935269923438634918454288176556644870427973106      3
Name: count, dtype: int64

In [107]:
df['withdrawal.l2_transaction.ticket.whitelisted'].value_counts()

withdrawal.l2_transaction.ticket.whitelisted
True    1524
Name: count, dtype: int64

### Multiple withdrawals in one hash:

In [112]:
df['withdrawal.l2_transaction.transaction_hash'].value_counts()

withdrawal.l2_transaction.transaction_hash
2ffa4514f53052b6dbdc8a5facd22e10be5cd3df687e6bd59aead7bb59538bdd    450
9645796b5153072f94c2c8390641da9603471eba0588408a5d414e9603f1754f    450
3a116245e9cb547dc101653d6e7014abf663d67ec378500291d77b5d2684d163    450
5a8f4126ebb02a99c815d7cd63cd25cc998633896f5e13e1c38e34b2af758a07    100
d461bb7ca25442f2688f279fb370833a1c0aea5b65aa00f59fee5b116cdcc79a     10
0e6079442f4278a0151e39f47048fc2abf0a7d63e420f6a5da8d6116f578ae12      5
f06a036fc3a1396ebe36ef80f9080748e5c58660162b8be0ab41ee29d1437dc6      3
9467e4fd7e4f1fe0e49088f49469d20c7cd598611c5ee9f90534fdea4945c67c      3
b00b86e3a978e2273530e89833d7aad00e453af5fdffae84be601c3b548aa6b9      3
9dca8aa2a43b2856788735ee2b734aedbc24079fa657d71b7adca76d06dd8594      3
99db7a6a30209965d791a71279ed0159ac6d053d2d00843f4c450b3081367b6b      1
be26bc31a16e334e479e0087e8f448b43e266d2446e3ec6faa668e026eb07c65      1
f1ba07529c3355cabd58e0c17bccf01d32e205149877c2d1113c1254391500d3      1
77ff272b3b1154028e04e

### What is `withdrawal.l2_transaction.transaction_index`?

In [113]:
df['withdrawal.l2_transaction.transaction_index'].value_counts()

withdrawal.l2_transaction.transaction_index
0.00    1043
1.00     477
2.00       4
Name: count, dtype: int64

### Cemented at:

In [127]:
df['withdrawal.outbox_message.cemented_at'].value_counts().sort_index()

withdrawal.outbox_message.cemented_at
2024-09-11T11:47:45+00:00      1
2024-09-13T20:56:01+00:00      2
2024-09-18T15:22:47+00:00      1
2024-09-21T13:30:45+00:00      2
2024-09-21T20:46:52+00:00      1
2024-09-21T21:01:52+00:00      2
2024-09-24T10:25:32+00:00      2
2024-09-24T11:55:32+00:00      5
2024-09-24T12:10:32+00:00      1
2024-09-24T12:40:32+00:00      1
2024-09-24T13:40:32+00:00      1
2024-09-25T08:10:32+00:00      1
2024-09-26T10:55:32+00:00      1
2024-09-27T05:10:32+00:00      2
2024-09-27T06:40:32+00:00      1
2024-09-27T15:10:32+00:00      2
2024-10-01T05:40:32+00:00      1
2024-10-02T02:55:32+00:00      1
2024-10-03T00:40:32+00:00      1
2024-10-03T19:40:32+00:00      2
2024-10-03T21:25:32+00:00     20
2024-10-03T21:40:32+00:00    100
2024-10-03T21:55:32+00:00    800
2024-10-03T22:10:32+00:00      1
2024-10-03T23:10:32+00:00      4
2024-10-03T23:25:32+00:00    106
2024-10-03T23:40:32+00:00    350
2024-10-08T09:28:50+00:00      1
2024-10-10T11:29:02+00:00      1
2024-

### Last withdrawals stats:

In [126]:
pd.DataFrame({
    'created_at': df['withdrawal.outbox_message.cemented_at'],
    'cemented_at': df['withdrawal.outbox_message.created_at'],
}).sort_index().dropna()[-10:]

,created_at,cemented_at
994,2024-10-03T23:25:32+00:00,2024-09-19T19:16:57+00:00
995,2024-10-03T21:55:32+00:00,2024-09-19T17:19:55+00:00
995,2024-10-03T23:40:32+00:00,2024-09-19T19:17:12+00:00
996,2024-10-03T23:25:32+00:00,2024-09-19T19:16:57+00:00
997,2024-10-03T21:55:32+00:00,2024-09-19T17:20:02+00:00
997,2024-10-03T23:40:32+00:00,2024-09-19T19:17:07+00:00
998,2024-10-03T21:55:32+00:00,2024-09-19T17:20:12+00:00
998,2024-10-03T23:40:32+00:00,2024-09-19T19:17:12+00:00
999,2024-10-03T21:55:32+00:00,2024-09-19T17:19:55+00:00
999,2024-10-03T23:40:32+00:00,2024-09-19T19:17:17+00:00


### Outbox initiators:
- [ ] why `withdrawal.l1_transaction.initiator` is only `MIDL.dev etherlink operator`?

In [141]:
df['withdrawal.l1_transaction.initiator'].value_counts()

withdrawal.l1_transaction.initiator
tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA    26
Name: count, dtype: int64

In [146]:
df['withdrawal.l1_transaction.sender'].value_counts()

withdrawal.l1_transaction.sender
tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn    26
Name: count, dtype: int64

### Executed withdrawals:

In [153]:
executed = df[ ~df['withdrawal.l1_transaction.sender'].isna() ]
executed[[
    'status',
    'withdrawal.l1_transaction.initiator',
    'withdrawal.l1_transaction.sender',
    'withdrawal.l2_transaction.l2_token.id',
    # 'withdrawal.l1_transaction.operation_hash'
]]

# executed.iloc[0]['withdrawal.l1_transaction.operation_hash']

,status,withdrawal.l1_transaction.initiator,withdrawal.l1_transaction.sender,withdrawal.l2_transaction.l2_token.id
534,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
535,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
536,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,baa233e2f62f45e9d91dacd3d6c6a57bc2cbc575
537,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
538,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
539,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
540,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,8e73ae3cf688fbd8368c99520d26f9ef1b4d3bca
541,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz
542,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,baa233e2f62f45e9d91dacd3d6c6a57bc2cbc575
543,FINISHED,tz1SJjg3biaXZft3KzUeNXTGSkDvzvHU6URA,tz1P2Jn2fQktnA4HHMkLRNy2ebuscUx7EUbn,xtz


### TODO: what are the closest outboxes to be executed?
### TODO: is there any outboxes that wasn't executed and impossible to execute?